# Validation library test



## Install the latest Vectice Python client library

In [1]:
from pathlib import Path
import sys

path_root = Path("./src/vectice").parents[1]
sys.path.append(str(path_root))
print(sys.path)

import vectice

print(path_root)

['/Users/bryandaversa/Vectice/sdk-python/perso', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python39.zip', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python3.9', '/Users/bryandaversa/.pyenv/versions/3.9.16/lib/python3.9/lib-dynload', '', '/Users/bryandaversa/Vectice/sdk-python/.venv/lib/python3.9/site-packages', '/Users/bryandaversa/Vectice/sdk-python/src', '.']
.


## Run your regular ML project

## Import library

In [1]:
# import piml
import pandas as pd

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder


# File system manangement
import os

# Suppress warnings
import warnings

warnings.filterwarnings("ignore")

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Feature engineering and Train test split

In [2]:
from sklearn.model_selection import train_test_split

# Training data
application_cleaned = pd.read_csv("/Users/bryandaversa/Vectice/GettingStarted/perso/Demo-Bank/application_train.csv")
app_train_feat, app_test_feat = train_test_split(application_cleaned, test_size=0.15, random_state=42)
# Separate the target variable from the testing set
target_variable = "TARGET"
app_test_feat_target = app_test_feat[target_variable]
app_test_feat = app_test_feat.drop(target_variable, axis=1)

# Print the shapes of the resulting dataframes
print("Training data shape: ", app_train_feat.shape)
print("Testing shape: ", app_test_feat.shape)
print("Testing target shape: ", app_test_feat_target.shape)

Training data shape:  (261384, 122)
Testing shape:  (46127, 121)
Testing target shape:  (46127,)


In [4]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train_feat:
    if app_train_feat[col].dtype == "object":
        # If 2 or fewer unique categories
        if len(list(app_train_feat[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train_feat[col])
            # Transform both training and testing data
            app_train_feat[col] = le.transform(app_train_feat[col])
            app_test_feat[col] = le.transform(app_test_feat[col])

            # Keep track of how many columns were label encoded
            le_count += 1

print("%d columns were label encoded." % le_count)


# one-hot encoding of categorical variables
app_train_feat = pd.get_dummies(app_train_feat)
app_test_feat = pd.get_dummies(app_test_feat)
train_labels = app_train_feat["TARGET"]

# Align the training and testing data, keep only columns present in both dataframes
app_train_feat, app_test_feat = app_train_feat.align(app_test_feat, join="inner", axis=1)

# Add the target back in
app_train_feat["TARGET"] = train_labels


# Create an anomalous flag column
app_train_feat["DAYS_EMPLOYED_ANOM"] = app_train_feat["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train_feat["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)


app_test_feat["DAYS_EMPLOYED_ANOM"] = app_test_feat["DAYS_EMPLOYED"] == 365243
app_test_feat["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
print("Training Features shape: ", app_train_feat.shape)
print("Testing Features shape: ", app_test_feat.shape)
print(
    "There are %d anomalies in the test data out of %d entries"
    % (app_test_feat["DAYS_EMPLOYED_ANOM"].sum(), len(app_test_feat))
)

3 columns were label encoded.
Training Features shape:  (261384, 244)
Testing Features shape:  (46127, 243)
There are 8287 anomalies in the test data out of 46127 entries


In [5]:
from sklearn.impute import SimpleImputer

# Drop the target from the training data
if "TARGET" in app_train_feat:
    train_no_missing = app_train_feat.drop(columns=["TARGET"])

features = list(train_no_missing.columns)

# Median imputation of missing values
imputer = SimpleImputer(strategy="median")
# Fit on the training data
imputer.fit(train_no_missing)

# Transform both training and testing data
train_no_missing = pd.DataFrame(imputer.transform(train_no_missing), columns=features).set_index("SK_ID_CURR")
test_no_missing = pd.DataFrame(imputer.transform(app_test_feat), columns=features).set_index("SK_ID_CURR")

# Build model

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=50, min_samples_leaf=0.01, n_jobs=-1)
features = list(train_no_missing.columns)
# Train on the training data
random_forest.fit(train_no_missing, train_labels)

# Extract feature importances
# feature_importance_values = random_forest.feature_importances_

# Make predictions on the test data
predictions = random_forest.predict_proba(test_no_missing)[:, 1]
prediction_absolte_threshold = (random_forest.predict_proba(test_no_missing)[:,1] >= 0.1).astype(bool) 

# Validation Library

## Connect to Vectice

In [14]:
import vectice
from vectice.models.validation import ValidationModel

connect = vectice.connect(
    api_token="oQjZ7xVOG.vaJwZbLMYD4WR2z8PN6moQjZ7xVOG35yKqpre7xl91nEkXB0Vd", host="https://qa.vectice.com"
)

phase = connect.phase("PHA-6406")
iteration = phase.create_iteration()

Welcome, bdaversa. You`re now successfully connected to Vectice.

To connect to a phase, copy your phase ID from your Vectice app and use connect.phase("PHA-XXXX")
To create your first iteration do iteration = connect.phase("PHA-XXXX").create_or_get_current_iteration()

If you are using a notebook you can call the help by using a Vectice returned object with the builtin notebook "?":
>> connection?

If you are using an IDE you can call the help() method on any object returned by Vectice:
>> help(connection)

For quick access to your default workspace in the Vectice web app, visit:
https://qa.vectice.com/browse/workspace/WSP-309
Phase 'Modeling' successfully retrieved.

For quick access to the Phase in the Vectice web app, visit:
https://qa.vectice.com/browse/phase/PHA-6406
New Iteration number 60 created.

For quick access to the Iteration in the Vectice web app, visit:
https://qa.vectice.com/browse/iteration/ITR-6694


## Tell us which Model and which test you want to run

In [21]:
validation_suite = ValidationModel(asset="MDV-4090",                        #The asset you want to run the test on
                                dataframe = train_no_missing,               #Your dataset
                                predictor = random_forest,                  #Your predictor
                                target = app_test_feat_target.values,       #The taregt
                                prediction = prediction_absolte_threshold,  #The prediction
                                tests = ["binary_full_suite"],              #The test suite you want to run
                                )


iteration.log(validation_suite)


Test: binary_tests successfully run
Model version 'MDV-4090' attachments successfully updated.
Existing model version: 'MDV-4090' of model: 'PHA-6786-knn' already linked to iteration: 60.
Link to iteration: https://qa.vectice.com/browse/iteration/ITR-6694



In [ ]:
validation_suite = ValidationModel(asset="MDV-4090",            #The asset you want to run the test on
                                dataframe = train_no_missing,   #Your dataset
                                predictor = random_forest,      #Your predictor
                                target = app_test_feat_target.values,       #The taregt
                                prediction = prediction_absolte_threshold,  #The prediction
                                prediction_proba = predictions,             #The predicted probabilities (Optional)
                                tests = ["binary_full_suite"],              #The test suite you want to run
                                )


iteration.log(validation_suite)

iteration.log(validation_suite)

In [40]:
test_accuracy_table(test_no_missing.values, app_test_feat_target.values, prediction_absolte, task_type="classification")

TypeError: can only concatenate list (not "NoneType") to list

In [38]:
app_test_feat_target.values

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
import shap
def explainability(
    dataframe: pd.DataFrame,
    target: List[int | float],
    prediction: List[int | float],
    prediction_proba: List[int | float] | None = None,
) -> dict[str, dict[str, any] | List[str]]:
    
    _ = target
    _ = prediction
    explainer = shap.Explainer(random_forest, train_no_missing)
    shap_values = explainer.shap_values(train_no_missing.head(1000))
    shap.summary_plot(shap_values[:,:,0],train_no_missing.head(1000), max_display=10, show=False)
    summary_plot_path = "SHAP_summary_plot.png"
    plt.savefig(summary_plot_path, bbox_inches='tight')
    plt.show()
    plt.close()
    # shap.plots.bar(shap_values[:,:,0], max_display=10, show=False)
    # feature_importance_path = "SHAP_feature_importance.png"
    # plt.savefig(feature_importance_path, bbox_inches='tight')
    # plt.show()
    # plt.close()

    #shap.plots.beeswarm(shap_values)
    #force_plot_path = "SHAP_force_plot.png"
    #plt.savefig(force_plot_path)
    #plt.close()

    return {"metric": {}, "property": {}, "table": [], "attachments": [summary_plot_path]}

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
explainer = shap.Explainer(random_forest, train_no_missing)
shap_values = explainer.shap_values(train_no_missing.head(1000))
shap.summary_plot(shap_values[:,:,0],train_no_missing.head(1000), max_display=8)

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,...,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
384575.0,0.0,1.0,0.0,2.0,207000.0,465457.5,52641.0,418500.0,0.009630,-13297.0,-762.0,-637.0,-4307.0,19.0,1.0,1.0,0.0,1.0,0.0,0.0,4.0,2.0,2.0,11.0,0.0,0.0,0.0,0.0,1.0,1.0,0.675878,0.604894,0.000527,0.0876,0.0763,0.9821,0.7552,0.0211,0.00,0.1379,0.1667,0.2083,0.0483,0.0756,0.0745,0.0000,0.0036,0.0840,0.0747,0.9816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214010.0,0.0,1.0,1.0,0.0,247500.0,1281712.5,48946.5,1179000.0,0.006852,-14778.0,-1141.0,-1610.0,-4546.0,11.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,3.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.430827,0.425351,0.712155,0.0753,0.0568,0.9970,0.9592,0.1326,0.08,0.0517,0.4167,0.2917,0.0735,0.0601,0.0844,0.0058,0.1118,0.0756,0.0566,0.9940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
142232.0,0.0,1.0,0.0,0.0,202500.0,495000.0,39109.5,495000.0,0.035792,-17907.0,-639.0,-2507.0,-1461.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,2.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.527239,0.531760,0.207964,0.0876,0.0763,0.9821,0.7552,0.0211,0.00,0.1379,0.1667,0.2083,0.0483,0.0756,0.0745,0.0000,0.0036,0.0840,0.0747,0.9816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [22]:
app_test_feat

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,...,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM
245895,384575,0,1,0,2,207000.0,465457.5,52641.0,418500.0,0.009630,-13297,-762.0,-637.0,-4307,19.0,1,1,0,1,0,0,4.0,2,2,11,0,0,0,0,1,1,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
98194,214010,0,1,1,0,247500.0,1281712.5,48946.5,1179000.0,0.006852,-14778,-1141.0,-1610.0,-4546,11.0,1,1,0,1,0,1,1.0,3,3,10,0,0,0,0,0,0,0.430827,0.425351,0.712155,0.0753,0.0568,0.9970,0.9592,0.1326,0.08,0.0517,0.4167,0.2917,0.0735,0.0601,0.0844,0.0058,0.1118,0.0756,0.0566,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,False
36463,142232,0,1,0,0,202500.0,495000.0,39109.5,495000.0,0.035792,-17907,-639.0,-2507.0,-1461,4.0,1,1,1,1,0,0,2.0,2,2,16,0,0,0,0,0,0,0.527239,0.531760,0.207964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False
249923,389171,0,0,1,0,247500.0,254700.0,24939.0,225000.0,0.046220,-19626,-6982.0,-11167.0,-3158,NaN,1,1,0,1,0,0,1.0,1,1,14,0,0,0,0,0,0,NaN,0.693521,0.614414,0.1320,0.0645,0.9846,NaN,NaN,0.16,0.0690,0.6250,NaN,NaN,NaN,0.1628,NaN,0.0022,0.1345,0.0670,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,False
158389,283617,0,0,1,0,112500.0,308133.0,15862.5,234000.0,0.018850,-20327,-1105.0,-7299.0,-494,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,0,0,0.654882,0.560690,0.636376,0.0619,0.0553,0.9717,NaN,NaN,0.00,0.1724,0.1667,NaN,0.0866,NaN,0.0749,NaN,0.0149,0.0630,0

In [ ]:
import IPython

IPython.__version__

'8.12.0'

In [ ]:
from vectice import autolog

autolog.config(api_token="3xKMWz5Qz.O2aoPrYGR7Z8O49Ddn6X03xKMWz5QzJpvwjkANmLVlBgMWeEyb", phase="PHA-4055")

Welcome, bdaversa. You`re now successfully connected to Vectice.
Phase '[Step 2] Autolog - Auto-document your work' successfully retrieved.

For quick access to the Phase in the Vectice web app, visit:
https://app.vectice.com/browse/phase/PHA-4055

Autolog is currently in Beta, and we are actively enhancing supported libraries, environments, and functionalities to provide an improved user experience.
For detailed information, supported libraries and environments please consult the documentation: https://api-docs.vectice.com/reference/vectice/autolog/


In [ ]:
# Paste your API key and Phase Id

import vectice

connect = vectice.connect(
    api_token="3xKMReqQz.O2aoPrYGR7Z8O49Ddn6X03xKMReqQzJpvwjkANmLVlBgMWeEyb", host="https://app.vectice.com"
)

phase = connect.phase("PHA-3855")
iteration = phase.create_iteration()

Welcome, bdaversa. You`re now successfully connected to Vectice.

To connect to a phase, copy your phase ID from your Vectice app and use connect.phase("PHA-XXXX")
To create your first iteration do iteration = connect.phase("PHA-XXXX").create_or_get_current_iteration()

If you are using a notebook you can call the help by using a Vectice returned object with the builtin notebook "?":
>> connection?

If you are using an IDE you can call the help() method on any object returned by Vectice:
>> help(connection)

For quick access to your default workspace in the Vectice web app, visit:
https://app.vectice.com/browse/workspace/WSP-124
Phase 'Fashion retail sails' successfully retrieved.

For quick access to the Phase in the Vectice web app, visit:
https://app.vectice.com/browse/phase/PHA-3855
New Iteration number 57 created.

For quick access to the Iteration in the Vectice web app, visit:
https://app.vectice.com/browse/iteration/ITR-1904
